# Projeto 1 - Ciência dos Dados

Nome: Ralph Michaan

Nome: Fábio Ferreira

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [79]:
%matplotlib inline
from emoji import UNICODE_EMOJI
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import *

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [80]:
import os

filename = 'iPhone.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPhone.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [81]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificação
0,feliz pelo iphone 11 mas o iphone 10 mds é mui...,0
1,quando eu estiver comprando o iphone 11 pro ma...,1
2,"não consegui comprar o iphone 11,imagina o 14",1
3,graças a deus vou compra meu iphone 11 pro max 🤩,1
4,"gente, qual a diferença pro iphone 11? kkkkkkk...",1


In [82]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@hyzerfps iphone 11 = fogao de 3 bocas\niphone...,0
1,@sirmaykon @uourah kkkkkkkkkkkkkkkkkkkkkk tem ...,0
2,@cnnbrbusiness @cnnbrasil por isso o mundo tá ...,0
3,rt @k3viyn: #appleevent esperando o iphone 12 ...,0
4,"é meus amigos, que fase! troquei gramado por u...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é um smartphone da Apple, lançado em setembro de 2019, por um preço um pouco mais baixo que suas variantes mais parrudas.

Consideramos relevante os tweets em que havia algum feedback sobre o produto, ou se a pessoa tinha vontade de ter, já os desconsiderados foram as piadas feitas com o valor, e os tweets que não se tratavam do iPhone 11 propriamente dito.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Limpando os tweets e pegando as frequências absolutas e relativas:

In [83]:
import re

def is_emoji(s):
    return s in UNICODE_EMOJI

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub('https', "", text_subbed)
    text_subbed = re.sub('\n', '', text_subbed)
    text_subbed = ''.join(' ' + char + ' ' if is_emoji(char) else char for char in text_subbed).strip()
    return text_subbed


def is_emoji(s):
    return s in UNICODE_EMOJI


Dos tweets relevantes:

In [84]:
train_rel_clean = train.loc[train["Classificação"] == 1, 'Treinamento'].apply(cleanup)
train_rel_clean

1      quando eu estiver comprando o iphone 11 pro ma...
2           não consegui comprar o iphone 11imagina o 14
3      graças a deus vou compra meu iphone 11 pro max  🤩
4      gente qual a diferença pro iphone 11 kkkkkkkk ...
7      @ginghinni7 @conservadorarv melhor do que comp...
                             ...                        
492    rt @juh_santos14 troco minha amiga por um ipho...
493          @rodsal200 um iphone 11 tá mais cara q isso
494    contando os dias pra pegar meu iphone 11 😍  😍 ...
495    rt @vick_417 e eu que quase comprei o iphone 1...
498    @wulfs1ghs ᅟᅟᅟ   ⠀/entrega uma caixa do iphone...
Name: Treinamento, Length: 233, dtype: object

In [85]:
train_rel_clean.astype(str)
train_rel_lower = train_rel_clean.str.lower()
train_rel = train_rel_lower.str.split()


train_rel_lista = []

for palavras in train_rel:
    train_rel_lista += palavras
serie_train_rel = pd.Series(train_rel_lista)
serie_train_rel

0          quando
1              eu
2         estiver
3       comprando
4               o
          ...    
4280           um
4281          ceo
4282        legal
4283          ᅟᅟᅟ
4284            ⠀
Length: 4285, dtype: object

In [86]:
serie_train_rel_absoluta = serie_train_rel.value_counts()
serie_train_rel_absoluta

iphone     255
11         228
o          179
um         122
eu          83
          ... 
—            1
saúde        1
monte        1
boa          1
demorei      1
Length: 1257, dtype: int64

E dos irrelevantes:

In [87]:
train_irrel_clean = train.loc[train["Classificação"] == 0, 'Treinamento'].apply(cleanup)
train_irrel_clean

0      feliz pelo iphone 11 mas o iphone 10 mds é mui...
5      fui aí à dois anos quando ofereci um iphone 11...
6      gosto muito do conceito do pobre com macbook p...
9      @gab_neres ce pedia meu rg pra ir no motel e e...
10     apareceu uma msg pro luan no google dizendo qu...
                             ...                        
490    eu só preciso saber quanto vai valer o iphone ...
491    um apoiador das causas da comunidade lgbtq pra...
496    e nem vou comentar o “comunistas” e o “anti ca...
497    rt @rannasoareez comprei o iphone 11 e aí inve...
499    preciso de um telefone novo alguém me dá um ip...
Name: Treinamento, Length: 267, dtype: object

In [88]:
train_irrel_clean.astype(str)
train_irrel_lower = train_irrel_clean.str.lower()
train_irrel = train_irrel_lower.str.split()

train_irrel_lista = []

for palavras in train_irrel:
    train_irrel_lista += palavras
serie_train_irrel = pd.Series(train_irrel_lista)
serie_train_irrel

0        feliz
1         pelo
2       iphone
3           11
4          mas
         ...  
5302        um
5303    iphone
5304        11
5305       por
5306     favor
Length: 5307, dtype: object

In [89]:
serie_train_irrel_absoluta = serie_train_irrel.value_counts()
serie_train_irrel_absoluta


iphone        290
11            262
o             177
um            135
de            127
             ... 
autoestima      1
confira         1
mimmeu          1
acorda          1
😶               1
Length: 1640, dtype: int64

In [90]:
train['Classificação'].value_counts(True)

0    0.534
1    0.466
Name: Classificação, dtype: float64

Criando a lista de palavras dos tweets e fazendo o smoothing:

In [91]:
total_rel_irrel = train_rel_lista + train_irrel_lista
qntd_total = len(total_rel_irrel)
qntd_total

9592

In [92]:
serie_rel_irrel = pd.Series(total_rel_irrel)

##### Criando o classificador:

Queremos calcular: 

- $P(relevante|frase)$: probabilidade de a frase ser relevante;
- $P(irrelevante|frase)$: probabilidade de a frase ser irrelevante;

A fórmula de Bayes é a seguinte:

$$P(relevante|frase) = \frac{P(frase|relevante) P(relevante)}{P(frase)}$$

Ela é análoga para os tweets irrelevantes, trocando tudo que é relevante por irrelevante.

Para calcularmos essa probabilidade, é necessário chegar nas seguintes informações:

- $P(frase|relevante)$: probabilidade da frase dado que é relevante;
- $P(frase|irrelevante)$: probabilidade da frase dado que é irrelevante;
- $P(relevante)$: probabilidade de ser relevante;
- $P(irrelevante)$: probabilidade de ser irrelevante;
- $P(frase)$: probabilidade de uma frase ocorrer na língua portuguesa.

In [93]:
#definindo o alfa e o V 
alfa = 1
V = 400000

In [94]:
#total de tweets relevantes e irrelevantes
total_relevante = len(serie_train_rel_absoluta) + (alfa*V)
total_irrelevante = len(serie_train_irrel_absoluta) + (alfa*V)

In [95]:
#total de palavras
total_p = (serie_train_rel_absoluta.sum() + serie_train_irrel_absoluta.sum())

In [96]:
#Probabilidade de ser relevante ou irrelevante
prob_rel = serie_train_rel_absoluta.sum() / total_p
prob_irrel = serie_train_irrel_absoluta.sum() / total_p

print(prob_rel)

0.44672643869891576


In [97]:
def classificador(tweet, planilha): #Função do classificador
    
    prob_tt_rel = 1.0 #probabilidade do tweet dado que ele é relevante
    prob_tt_irrel = 1.0 #probabilidade do tweet dado que ele é irrelevante
    i = 0
    
    for tt in tweet:
        for a in tt:

            if a in serie_train_rel_absoluta:
                prob_a_dado_rel = (serie_train_rel_absoluta[a] + alfa)/ total_relevante
                prob_tt_rel *= prob_a_dado_rel
            else: 
                prob_a_dado_rel = alfa / total_relevante
                prob_tt_rel *= prob_a_dado_rel
                
            if a in serie_train_irrel_absoluta:
                prob_a_dado_irrel = (serie_train_irrel_absoluta[a] + alfa)/ total_irrelevante
                prob_tt_irrel *= prob_a_dado_irrel
            else:
                prob_a_dado_irrel = alfa / total_irrelevante
                prob_tt_irrel *= prob_a_dado_irrel
                
        prob_rel_dado_tt = prob_tt_rel * prob_rel
        prob_irrel_dado_tt = prob_tt_irrel * prob_irrel
    
        if prob_rel_dado_tt > prob_irrel_dado_tt:
            planilha.iloc[i,2] = 1
        elif prob_irrel_dado_tt > prob_rel_dado_tt:
            planilha.iloc[i,2] = 0
        
        i +=1
        prob_tt_rel = 1.0 #probabilidade do tweet dado que ele é relevante
        prob_tt_irrel = 1.0 #probabilidade do tweet dado que ele é irrelevante    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

Limpando e pegando as frequências relativas e absolutas dos tweets relevantes:

## Rodando o classificador com os tweets de treinamento

In [98]:
train_tts = []
train_cleaner = train['Treinamento'].apply(cleanup)


In [99]:
for tt in train_cleaner:
    train_tts.append(tt.split())

In [100]:
train['Classificador'] = 'X'

In [101]:
classificador(train_tts, train)
train

,Treinamento,Classificação,Classificador
0,feliz pelo iphone 11 mas o iphone 10 mds é mui...,0,0
1,quando eu estiver comprando o iphone 11 pro ma...,1,1
2,"não consegui comprar o iphone 11,imagina o 14",1,1
3,graças a deus vou compra meu iphone 11 pro max 🤩,1,1
4,"gente, qual a diferença pro iphone 11? kkkkkkk...",1,0
...,...,...,...
495,rt @vick_417: e eu que quase comprei o iphone ...,1,0
496,e nem vou comentar o “comunistas” e o “anti ca...,0,0
497,rt @rannasoareez: comprei o iphone 11 e aí inv...,0,0
498,@wulfs1ghs ᅟᅟᅟ ⠀\n/entrega uma caixa do ipho...,1,0


In [102]:
pd.crosstab(train['Classificação'],train['Classificador'],normalize = 'all')

Classificador,0,1
Classificação,,
0,0.526,0.008
1,0.126,0.340


In [113]:
train["Classificação"].value_counts(True)

0    0.534
1    0.466
Name: Classificação, dtype: float64

## Rodando o classificador com os tweets de teste


In [104]:
teste_rel_clean = test.loc[test["Relevância"] == 1, 'Teste'].apply(cleanup)
teste_rel_clean

4      é meus amigos que fase troquei gramado por um ...
5      meu pai comprou um iphone 11 e eu ainda achei ...
7               vou pegar o iphone 11 final do ano  😍  😍
10     a apple já criou o iphone 12 enquanto eu acabe...
12     eu babava pelo carregador turbo do motorola ma...
                             ...                        
223    rt @fernandavsntss eu to muito triste e soment...
225      deixem me ter iphone 11 vão ver só snaps a cair
229    @renatinhapoqui n consigo fazer meu ead pq meu...
234    bateria do iphone 11 é tudooo ainda nem carreg...
237                        @_ilipe quero um iphone 11  🤔
Name: Teste, Length: 89, dtype: object

In [105]:
teste_rel_clean.astype(str)
teste_rel_lower = teste_rel_clean.str.lower()
teste_rel = teste_rel_lower.str.split()


teste_rel_lista = []

for palavras in teste_rel:
    teste_rel_lista += palavras
serie_teste_rel = pd.Series(teste_rel_lista)
serie_teste_rel

0            é
1         meus
2       amigos
3          que
4         fase
         ...  
1481     quero
1482        um
1483    iphone
1484        11
1485         🤔
Length: 1486, dtype: object

In [106]:
serie_teste_rel_absoluta = serie_teste_rel.value_counts()
serie_teste_rel_absoluta

iphone          92
11              90
o               67
um              45
do              28
                ..
lugar            1
@placewantsz     1
ufa              1
vender           1
@taylaiss        1
Length: 557, dtype: int64

In [107]:
serie_teste_rel_relativa = serie_train_rel.value_counts(True)
serie_teste_rel_relativa

iphone     0.059510
11         0.053209
o          0.041774
um         0.028471
eu         0.019370
             ...   
—          0.000233
saúde      0.000233
monte      0.000233
boa        0.000233
demorei    0.000233
Length: 1257, dtype: float64

E dos irrelevantes:

In [108]:
teste_irrel_clean = test.loc[test["Relevância"] == 0, 'Teste'].apply(cleanup)
teste_irrel_clean

0      @hyzerfps iphone 11 = fogao de 3 bocasiphone 1...
1      @sirmaykon @uourah kkkkkkkkkkkkkkkkkkkkkk tem ...
2      @cnnbrbusiness @cnnbrasil por isso o mundo tá ...
3      rt @k3viyn appleevent esperando o iphone 12 la...
6                                 era só um iphone 11  🤧
                             ...                        
231    eu nunca me esqueço do dia q fui tapeada num s...
232                 ganhar um iphone 11 //tco/coxrl1zkza
233    rezando pra que tenha chegado película pro iph...
235    o candidato de igt bancando o populista matuto...
236    @junior_onda vamos pegar o iphone 11 do nosso ...
Name: Teste, Length: 149, dtype: object

In [109]:
teste_irrel_clean.astype(str)
teste_irrel_lower = teste_irrel_clean.str.lower()
teste_irrel = teste_irrel_lower.str.split()


teste_irrel_lista = []

for palavras in teste_irrel:
    teste_irrel_lista += palavras
serie_teste_irrel = pd.Series(teste_irrel_lista)
serie_teste_irrel

0                 @hyzerfps
1                    iphone
2                        11
3                         =
4                     fogao
               ...         
2780                     do
2781                  nosso
2782               parceiro
2783       @brendonricardo2
2784    kkkkkkkkkkkkkkkkkkk
Length: 2785, dtype: object

In [110]:
serie_teste_irrel_absoluta = serie_teste_irrel.value_counts()
serie_teste_irrel_absoluta

iphone        164
11            146
o              91
e              57
que            57
             ... 
sobre           1
intagram        1
😁               1
dps             1
classifica      1
Length: 983, dtype: int64

In [111]:
serie_teste_irrelevante_relativa = serie_train_irrelevante.value_counts(True)
serie_teste_irrelevante_relativa

iphone                   0.059257
11                       0.050552
o                        0.033646
um                       0.027787
de                       0.024104
                           ...   
https//tco/tqbbtqbgxo    0.000167
pronta                   0.000167
free                     0.000167
700                      0.000167
😶                        0.000167
Length: 1750, dtype: float64

*Implementando o Classificador*

In [117]:
test_tts = []
test_cleaner = test['Teste'].apply(cleanup)

In [118]:
for tt in test_cleaner:
    test_tts.append(tt.split())

In [125]:
test['Classificador'] = 'X'

In [126]:
classificador(test_tts, test)
test

,Teste,Relevância,Classificador
0,@hyzerfps iphone 11 = fogao de 3 bocas\niphone...,0,0
1,@sirmaykon @uourah kkkkkkkkkkkkkkkkkkkkkk tem ...,0,0
2,@cnnbrbusiness @cnnbrasil por isso o mundo tá ...,0,0
3,rt @k3viyn: #appleevent esperando o iphone 12 ...,0,0
4,"é meus amigos, que fase! troquei gramado por u...",1,0
...,...,...,...
233,rezando pra que tenha chegado película pro iph...,0,0
234,bateria do iphone 11 é tudooo! ainda nem carre...,1,1
235,o candidato de igt bancando o populista matuto...,0,0
236,@junior_onda vamos pegar o iphone 11 do nosso ...,0,1


In [124]:
pd.crosstab(test['Relevância'],test['Classificador'],normalize = 'all')

Classificador,0,1
Relevância,,
0,0.575630,0.050420
1,0.205882,0.168067


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**